In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import json
import pickle
from datetime import date, timedelta

In [ ]:
_kamers = open("kamers.json", "r")
kamers = json.loads(_kamers.read())
_kamers.close()

_devices = open("devices.p", "rb")
devices = pickle.load(_devices)
_devices.close()

In [ ]:
one = devices['Huis']['Electriciteit_opwek_dak']
# The monitoring device I'm using _always_ reads either 0.1 or 0.2 Wh / 15 minutes
# , so only retain values above that amount
one = one[one.apply(lambda x: x['value'] > 0.2, axis=1)]

two = devices['Huis']['Electriciteit_opwek_garage']

cm_low_in = devices['Huis']['Electriciteit_gebruik_laag']
cm_high_in = devices['Huis']['Electriciteit_gebruik_hoog']

cm_low_out = devices['Huis']['Electriciteit_teruglevering_laag']
cm_high_out = devices['Huis']['Electriciteit_teruglevering_hoog']


def merge(a, b):
    c = pd.merge(a, b, left_index=True, right_index=True).sum(axis=1).to_frame()
    return c.rename(columns={c.columns[0]: 'value' })

prod = merge(one, two)
usage = merge(cm_low_in, cm_high_in)
feedback = merge(cm_low_out, cm_high_out)

years = [ str(y) for y in list(one.index.year.unique()) ]

sns.set(rc={'figure.figsize':(18, 6)})

In [ ]:
gas = devices['Huis']['Gas_gebruik']

In [ ]:
# From https://stackoverflow.com/a/41259922
def align_yaxis(ax1, ax2):
    """Align zeros of the two axes, zooming them out by same ratio"""
    axes = (ax1, ax2)
    extrema = [ax.get_ylim() for ax in axes]
    tops = [extr[1] / (extr[1] - extr[0]) for extr in extrema]
    # Ensure that plots (intervals) are ordered bottom to top:
    if tops[0] > tops[1]:
        axes, extrema, tops = [list(reversed(l)) for l in (axes, extrema, tops)]

    # How much would the plot overflow if we kept current zoom levels?
    tot_span = tops[1] + 1 - tops[0]

    b_new_t = extrema[0][0] + tot_span * (extrema[0][1] - extrema[0][0])
    t_new_b = extrema[1][1] - tot_span * (extrema[1][1] - extrema[1][0])
    axes[0].set_ylim(extrema[0][0], b_new_t)
    axes[1].set_ylim(t_new_b, extrema[1][1])
    return axes

# https://stackoverflow.com/a/46901839
def align_yaxis_np(axes):
    """Align zeros of the two axes, zooming them out by same ratio"""
    axes = np.array(axes)
    extrema = np.array([ax.get_ylim() for ax in axes])

    # reset for divide by zero issues
    for i in range(len(extrema)):
        if np.isclose(extrema[i, 0], 0.0):
            extrema[i, 0] = -1
        if np.isclose(extrema[i, 1], 0.0):
            extrema[i, 1] = 1

    # upper and lower limits
    lowers = extrema[:, 0]
    uppers = extrema[:, 1]

    # if all pos or all neg, don't scale
    all_positive = False
    all_negative = False
    if lowers.min() > 0.0:
        all_positive = True

    if uppers.max() < 0.0:
        all_negative = True

    if all_negative or all_positive:
        # don't scale
        return

    # pick "most centered" axis
    res = abs(uppers+lowers)
    min_index = np.argmin(res)

    # scale positive or negative part
    multiplier1 = abs(uppers[min_index]/lowers[min_index])
    multiplier2 = abs(lowers[min_index]/uppers[min_index])

    for i in range(len(extrema)):
        # scale positive or negative part based on which induces valid
        if i != min_index:
            lower_change = extrema[i, 1] * -1*multiplier2
            upper_change = extrema[i, 0] * -1*multiplier1
            if upper_change < extrema[i, 1]:
                extrema[i, 0] = lower_change
            else:
                extrema[i, 1] = upper_change

        # bump by 10% for a margin
        extrema[i, 0] *= 1.1
        extrema[i, 1] *= 1.1

    # set axes limits
    [axes[i].set_ylim(*extrema[i]) for i in range(len(extrema))]

def chart_timespan(begin, end, resample, label):
    begin = begin.strftime('%Y-%m-%d')
    end = end.strftime('%Y-%m-%d')
    
    df = usage[begin:end].index.to_frame().drop(['_from'], axis=1)

    df['Productie'] = prod[begin:end]['value'].apply(lambda x: x*-1)
    df['Consumptie'] = usage[begin:end]['value']
    df['Teruglevering'] = feedback[begin:end]['value'].apply(lambda x: x*-1)
    df = df.fillna(0)
    df['Netto gebruik'] = df.apply(lambda r: r['Consumptie'] - ((r['Productie']) - r['Teruglevering']), axis=1)
    
    #rt = merge(usage, feedback)[begin:end].cumsum(axis=0).apply(lambda x: x / 1000)
    rt = merge(df['Consumptie'], df['Teruglevering'])[begin:end].cumsum(axis=0).apply(lambda x: x / 1000)
    cumprod = df['Productie'][begin:end].cumsum(axis=0).apply(lambda x: x / 1000).to_frame()
    cum_net_cons = df['Consumptie'][begin:end].cumsum(axis=0).apply(lambda x: x / 1000).to_frame()
    
    tmp = df['Productie'].copy().apply(lambda x: x*-1)
    own_usage = merge(tmp, df['Teruglevering'])
    cum_own_usage = own_usage[begin:end].cumsum(axis=0).apply(lambda x: x / 1000)
    cumcons = merge(df['Consumptie'], own_usage)[begin:end].cumsum(axis=0).apply(lambda x: x / 1000)
    
    df = resample(df)
    rt = resample(rt)
    
    details = df.plot.area(stacked = False)
    details.set_xlabel('Tijd van ' + begin + ' tot ' + end)
    details.set_ylabel(label)
    
    running_total = rt['value'].plot(ax=details, style='g-', secondary_y=True, label='Running total')
    running_total.set_ylabel('Running totals in kWh')
    running_total.legend()
        
    cumulative_production = cumprod['Productie'].plot(ax=details, style='b-', secondary_y=True, label='Σ production')
    cumulative_production.legend()
    
    cumulative_net_consumption = cum_net_cons['Consumptie'].plot(ax=details, style='k-', secondary_y=True, label='Σ net consumption')
    cumulative_net_consumption.legend()
    
    cumulative_own_usage = cum_own_usage['value'].plot(ax=details, style='m-', secondary_y=True, label='Σ own usage')
    cumulative_own_usage.legend()

    cumulative_consumption = cumcons['value'].plot(ax=details, style='r-', secondary_y=True, label='Σ consumption')
    cumulative_consumption.legend(bbox_to_anchor=(0.15, 0.95))

    # return align_yaxis(details, running_total)
    return align_yaxis_np([details, running_total, cumulative_production, cumulative_net_consumption, cumulative_own_usage, cumulative_consumption])

In [ ]:
today = date.today()
yesterday = today - timedelta(days=1)
week = today - timedelta(days=7)
month = today - timedelta(days=30)
quartile = today - timedelta(days=90)
start_of_year = date(date.today().year, 1, 1)

chart_timespan(yesterday, today, lambda df: df, 'Wh / 15 minuten')
chart_timespan(week, today, lambda df: df.resample('H').sum(), 'Wh')
chart_timespan(month, today, lambda df: df.resample('24H').sum(), 'Wh')
chart_timespan(quartile, today, lambda df: df.resample('24H').sum(), 'Wh')

In [ ]:
for year in years[1:]:
    a = date(int(year), 1, 1)
    b = date(int(year), 12, 31)
    chart_timespan(a, b, lambda df: df.resample('168H').sum().apply(lambda x: x / 1000), 'kWh / week')

In [ ]:
def compare_years(source, years, x_axis = lambda d: d.index.dayofyear):
    fig, ax = plt.subplots()
    for year in years:
        x = source.loc[year, 'value']
        x = x.map(lambda x: x / 1000, na_action=None)
        ax.plot(x_axis(x), x, marker='x', linestyle='-', linewidth=0.5, label='kWh')
    ax.set_ylabel('kWh')
    ax.legend()

def compare_years_cumsum(source, years, xlabel):
    fig, ax = plt.subplots()
    source = source.fillna(0)
    for year in years:
        x = source.loc[year]
        x = x.cumsum(axis=0)
        x = x.map(lambda x: x / 1000, na_action=None)
        lbl = year + ' end: ' + str(round(x.iloc[-1], 2)) + ' kWh'
        ax.plot(x.index.dayofyear, x, marker='.', linestyle='-', linewidth=0.5, label=lbl)
    ax.set_ylabel('kWh')
    ax.set_xlabel(xlabel)
    ax.legend()
    
def compare_years_cumsum_gas(source, years, xlabel):
    fig, ax = plt.subplots()
    source = source.fillna(0)
    for year in years:
        x = source.loc[year]
        x = x.cumsum(axis=0)
        lbl = year + ' end: ' + str(round(x.iloc[-1], 2)) + ' m3'
        ax.plot(x.index.dayofyear, x, marker='.', linestyle='-', linewidth=0.5, label=lbl)
    ax.set_ylabel('m3')
    ax.set_xlabel(xlabel)
    ax.legend()

In [ ]:
compare_years_cumsum(one['value'], years, 'Huis')
# The first 2 years there was no data anyways
compare_years_cumsum(two['value'], years[2:], 'Garage')
compare_years_cumsum(prod['value'], years, 'Beide')

In [ ]:
compare_years_cumsum_gas(gas['value'], years, 'Gas')

In [ ]:
def scatter_plot(df, noise):
    df['day'] = df.index.dayofyear
    df['hour'] = df.index.hour
    df = df[df.apply(lambda x: x['value'] > noise, axis=1)]
    df.plot.scatter(x='day', y='hour', c='value', cmap='viridis')

In [ ]:
scatter_plot(one, noise=10)

In [ ]:
scatter_plot(two, noise=10)

In [ ]:
scatter_plot(prod, noise=10)

In [ ]:
scatter_plot(gas, noise=0.1)